### Imports, directory, and parser set-up

In [1]:
import argparse, subprocess, datetime, os, pdb, sys

In [2]:
# Get the HOME environment variable, with a fallback to the user's directory on Windows
home_dir = os.getenv("HOME") or os.path.expanduser("~")

In [3]:
parser = argparse.ArgumentParser(description='This script takes a model, and apply this model to new video clips')

### Input data

In [4]:
# 1. Directory of video clips
parser.add_argument('--Input_videos_directory',
                    type = str, 
                    default = os.path.join(home_dir,'data/labeled_videos'),
                    required = False, 
                    help = 'Name of directory to hold all video clips')


args, unknown = parser.parse_known_args()
if not os.path.exists(args.Input_videos_directory):
    print("path does not exist, creating path")
    os.makedirs(args.Input_videos_directory)

print(f"Using input videos directory: {args.Input_videos_directory}")

Using input videos directory: /home/hice1/kpatherya3/data/labeled_videos


In [5]:
# 2. Mapping of video clips to project
parser.add_argument('--Videos_to_project_file',
                    type = str, 
                    default = os.path.join(home_dir,'data/project_animal_map.csv'),
                    help = 'Project each animal belongs to')

args, unknown = parser.parse_known_args()
if not os.path.exists(args.Videos_to_project_file):
    print("path does not exist, creating path")
    os.makedirs(args.Videos_to_project_file)

print(f"Using videos to project file directory: {args.Videos_to_project_file}")

Using videos to project file directory: /home/hice1/kpatherya3/data/project_animal_map.csv


In [6]:
# 3. Saving the previous training's model results
parser.add_argument('--Trained_model',
                    default = os.path.join(home_dir,'data/save_50.pth'),
                    type = str,
                    help = 'Save data (.pth) of previous training')

args, unknown = parser.parse_known_args()
if not os.path.exists(args.Trained_model):
    print("path does not exist, creating path")
    os.makedirs(args.Trained_model)

print(f"Using trained model directory: {args.Trained_model}")

Using trained model directory: /home/hice1/kpatherya3/data/save_50.pth


In [7]:
# 4. JSON file previously used for training
parser.add_argument('--Trained_categories',
                    type = str, 
                    default = os.path.join(home_dir,'data/source.json'),
                    help = 'JSON file previously used for training')

args, unknown = parser.parse_known_args()
if not os.path.exists(args.Trained_categories):
    print("path does not exist, creating path")
    os.makedirs(args.Trained_categories)

print(f"Using trained categories directory: {args.Trained_categories}")

path does not exist, creating path
Using trained categories directory: /home/hice1/kpatherya3/data/source.json


In [8]:
# 5. Log file used for training
parser.add_argument('--Training_options',
                    type = str, 
                    default = os.path.join(home_dir,'data/log_test'),
                    help = 'log file in training')

args, unknown = parser.parse_known_args()
if not os.path.exists(args.Training_options):
    print("path does not exist, creating path")
    os.makedirs(args.Trained_options)

print(f"Using trained options directory: {args.Training_options}")

Using trained options directory: /home/hice1/kpatherya3/data/log_test


In [9]:
# 6. Output CSV that details the confidence and label for each video clip 
parser.add_argument('--Output_file',
                    type = str, 
                    default = os.path.join(home_dir,'data/output.csv'),
                    help = 'CSV file that keeps the confidence and label for each video clip')

args, unknown = parser.parse_known_args()
if not os.path.exists(args.Output_file):
    print("path does not exist, creating path")
    os.makedirs(args.Output_file)

print(f"Using output file directory: {args.Output_file}")

path does not exist, creating path
Using output file directory: /home/hice1/kpatherya3/data/output.csv


### Temporary directory that would be deleted at the end of the analysis

In [10]:
# 10. Location of temporary clips
parser.add_argument('--Temporary_clips_directory',
                    default = os.path.join(home_dir,'data/clips_temp'),
                    type = str, 
                    required = False, 
                    help = 'Location for temp clips to be stored')

args, unknown = parser.parse_known_args()
if not os.path.exists(args.Temporary_clips_directory):
    print("path does not exist, creating path")
    os.makedirs(args.Temporary_clips_directory)

print(f"Using temporary clips directory: {args.Temporary_clips_directory}")

path does not exist, creating path
Using temporary clips directory: /home/hice1/kpatherya3/data/clips_temp


In [11]:
# 11. Location of temporary files
parser.add_argument('--Temporary_output_directory',
                    default = os.path.join(home_dir,'data/intermediate_temp'),
                    type = str, 
                    required = False, 
                    help = 'Location for temp files to be stored')

args, unknown = parser.parse_known_args()
if not os.path.exists(args.Temporary_output_directory):
    print("path does not exist, creating path")
    os.makedirs(args.Temporary_output_directory)

print(f"Using temporary clips directory: {args.Temporary_output_directory}")

path does not exist, creating path
Using temporary clips directory: /home/hice1/kpatherya3/data/intermediate_temp


### Arguments that don't require a file to be passed in

In [12]:
# 7. Purpose of the script
parser.add_argument('--Purpose',
                    type = str, 
                    default = 'classify',
                    help = 'classify is the only function for this script for now')

# 8. Batch size for the model
parser.add_argument('--batch_size', 
                    default=13, 
                    type=int, help='Batch Size')

# 9. Number of workers
parser.add_argument('--n_threads',
                    default=5,
                    type=int,
                    help='Number of threads for multi-thread loading')

# 10. GPU card to use
parser.add_argument('--gpu_card',
                    default='1',
                    type=str,
                    help='gpu card to use')

_StoreAction(option_strings=['--gpu_card'], dest='gpu_card', nargs=None, const=None, default='1', type=<class 'str'>, choices=None, required=False, help='gpu card to use', metavar=None)

In [13]:
# Similar parameters, but these are for the dataloader

# 12. The sample duration of each inputted clip
parser.add_argument('--sample_duration',
                    default=96,
                    type=int,
                    help='Temporal duration of inputs')

# 13. Standardized height and width of inputs                    
parser.add_argument('--sample_size',
                    default=120,
                    type=int,
                    help='Height and width of inputs')

_StoreAction(option_strings=['--sample_size'], dest='sample_size', nargs=None, const=None, default=120, type=<class 'int'>, choices=None, required=False, help='Height and width of inputs', metavar=None)

In [14]:
# 14. Parameters for the optimizer
parser.add_argument('--learning_rate', default=0.1, type=float, help='Initial learning rate (divided by 10 while training by lr scheduler)')
parser.add_argument('--momentum', default=0.9, type=float, help='Momentum')
parser.add_argument('--dampening', default=0.9, type=float, help='dampening of SGD')
parser.add_argument('--weight_decay', default=1e-5, type=float, help='Weight Decay')
parser.add_argument('--nesterov', action='store_true', help='Nesterov momentum')
parser.set_defaults(nesterov = False)
parser.add_argument('--optimizer', default='sgd', type=str, help='Currently only support SGD')
parser.add_argument('--lr_patience', default=10, type=int, help='Patience of LR scheduler. See documentation of ReduceLROnPlateau.')
parser.add_argument('--resnet_shortcut', default='B', help='Shortcut type of resnet (A | B)')

_StoreAction(option_strings=['--resnet_shortcut'], dest='resnet_shortcut', nargs=None, const=None, default='B', type=None, choices=None, required=False, help='Shortcut type of resnet (A | B)', metavar=None)

In [15]:
# 15. Parameters specific for training from scratch
parser.add_argument('--n_classes', default=10, type=int)

_StoreAction(option_strings=['--n_classes'], dest='n_classes', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [16]:
# The code below is needed if you're running this script from a Jupyter notebook
if 'ipykernel_launcher' in sys.argv[0]:
    args, unknown = parser.parse_known_args()
else:
    args = parser.parse_args()